#  Data Wrangling: clean, Transform, Merge, Reshape

In [2]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4, threshold=500)
pd.options.display.max_rows = 100

In [3]:
%matplotlib inline

In [4]:
from IPython.core.interactiveshell import InteractiveShell

In [5]:
InteractiveShell.ast_node_interactivity = 'all'

## Combining and merging data sets

### Database-style DataFrame merges

In [6]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df2 = DataFrame({'key': ['a', 'b', 'd'],
                 'data2': range(3)})
df1
df2

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


,data2,key
0,0,a
1,1,b
2,2,d


In [7]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [8]:
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [9]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df4 = DataFrame({'rkey': ['a', 'b', 'd'],
                 'data2': range(3)})

df3
df4

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


,data2,rkey
0,0,a
1,1,b
2,2,d


In [10]:
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [11]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [12]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                 'data1': range(6)})
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                 'data2': range(5)})
df1
df2

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [13]:
pd.merge(df1, df2, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [14]:
pd.merge(df1, df2, how='inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [15]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})

left
right

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [16]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [17]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [18]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on index

In [19]:
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                  'value': range(6)})
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

left1
right1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


In [20]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [21]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [22]:
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                   'key2': [2000, 2001, 2002, 2001, 2002],
                   'data': np.arange(5.)})
righth = DataFrame(np.arange(12).reshape((6, 2)),
                   index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                          [2001, 2000, 2000, 2000, 2001, 2002]],
                   columns=['event1', 'event2'])

lefth
righth

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [23]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [24]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
        right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


In [25]:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
                 columns=['Ohio', 'Nevada'])
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                   index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])

left2
right2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [26]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [27]:
left2
right2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [29]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [31]:
left1
right1
left1.join(right1, on='key')

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [32]:
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                    index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon'])

left2
right2
another

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [33]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [34]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


## Fancy indexing equivalents: take and put

In [46]:
arr = np.arange(10) *100
inds = [7, 1, 2, 6]
arr[inds]

array([700, 100, 200, 600])

In [37]:
arr.take(inds)
arr.put(inds, 42)
arr
arr.put(inds, [40, 41, 42, 43])
arr

array([42, 42, 42, 42])

array([  0,  42,  42, 300, 400, 500,  42,  42, 800, 900])

array([  0,  41,  42, 300, 400, 500,  43,  40, 800, 900])

In [38]:
inds = [2, 0, 2, 1]
arr= randn(2, 4)
arr
arr.take(inds, axis=1)

array([[-0.2047,  0.4789, -0.5194, -0.5557],
       [ 1.9658,  1.3934,  0.0929,  0.2817]])

array([[-0.5194, -0.2047, -0.5194,  0.4789],
       [ 0.0929,  1.9658,  0.0929,  1.3934]])

## Broadcasting

In [48]:
arr = np.arange(5)
arr
arr * 4

array([0, 1, 2, 3, 4])

array([ 0,  4,  8, 12, 16])

In [50]:
a = np.array([1.0, 2.0, 3.0])
b = np.array([2.0, 2.0, 2.0])
a*b

array([ 2.,  4.,  6.])

In [51]:
a = np.array([1.0, 2.0, 3.0])
b = 2.0
# a와 b의 차원이 다르지만, 파이썬 내부에서 a와 b의 차원과 길이를 맞춰서
# 연산 수행(실제복사 X, 메모리 사용과 계산이 효율적 O)
a*b

array([ 2.,  4.,  6.])

In [54]:
x = np.arange(4); x
xx = x.reshape(4, 1); xx
y = np.ones(5); y
z = np.ones((3, 4)); z

array([0, 1, 2, 3])

array([[0],
       [1],
       [2],
       [3]])

array([ 1.,  1.,  1.,  1.,  1.])

array([[ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.]])

In [55]:
x.shape
y.shape

(4,)

(5,)

In [56]:
x+y

ValueError: operands could not be broadcast together with shapes (4,) (5,) 

In [57]:
xx.shape
y.shape

(4, 1)

(5,)

In [58]:
(xx+y).shape

(4, 5)

In [59]:
xx + y

array([[ 1.,  1.,  1.,  1.,  1.],
       [ 2.,  2.,  2.,  2.,  2.],
       [ 3.,  3.,  3.,  3.,  3.],
       [ 4.,  4.,  4.,  4.,  4.]])

In [60]:
x.shape
z.shape

(4,)

(3, 4)

In [61]:
(x + z).shape

(3, 4)

In [62]:
x + z

array([[ 1.,  2.,  3.,  4.],
       [ 1.,  2.,  3.,  4.],
       [ 1.,  2.,  3.,  4.]])

In [63]:
arr = randn(4, 3)
arr
arr.mean(0)

array([[ 0.769 ,  1.2464,  1.0072],
       [-1.2962,  0.275 ,  0.2289],
       [ 1.3529,  0.8864, -2.0016],
       [-0.3718,  1.669 , -0.4386]])

array([ 0.1135,  1.0192, -0.301 ])

In [64]:
demeaned = arr - arr.mean(0)
demeaned

array([[ 0.6556,  0.2272,  1.3082],
       [-1.4097, -0.7442,  0.5299],
       [ 1.2394, -0.1328, -1.7006],
       [-0.4853,  0.6498, -0.1375]])

In [65]:
demeaned.mean(0)

array([ -2.7756e-17,  -1.6653e-16,  -2.7756e-17])

In [66]:
arr
row_means = arr.mean(1)
row_means

array([[ 0.769 ,  1.2464,  1.0072],
       [-1.2962,  0.275 ,  0.2289],
       [ 1.3529,  0.8864, -2.0016],
       [-0.3718,  1.669 , -0.4386]])

array([ 1.0075, -0.2641,  0.0792,  0.2862])

In [67]:
arr - row_means # arr(4x3), row_means(1x4) 이므로  3, 4를 맟출 수 없으므로 에러

ValueError: operands could not be broadcast together with shapes (4,3) (4,) 

In [68]:
row_means.reshape((4, 1))

array([[ 1.0075],
       [-0.2641],
       [ 0.0792],
       [ 0.2862]])

In [70]:
demeaned = arr - row_means.reshape((4, 1))
demeaned

array([[ -2.3853e-01,   2.3889e-01,  -3.5953e-04],
       [ -1.0321e+00,   5.3910e-01,   4.9302e-01],
       [  1.2737e+00,   8.0719e-01,  -2.0809e+00],
       [ -6.5805e-01,   1.3828e+00,  -7.2477e-01]])

In [73]:
demeaned.mean(1)
demeaned.mean(0)

array([  0.0000e+00,   3.7007e-17,   0.0000e+00,   7.4015e-17])

array([-0.1638,  0.742 , -0.5782])

###  Braodcasting over other axes

In [74]:
arr - arr.mean(1)

ValueError: operands could not be broadcast together with shapes (4,3) (4,) 

In [75]:
arr - arr.mean(1).reshape((4, 1))

array([[ -2.3853e-01,   2.3889e-01,  -3.5953e-04],
       [ -1.0321e+00,   5.3910e-01,   4.9302e-01],
       [  1.2737e+00,   8.0719e-01,  -2.0809e+00],
       [ -6.5805e-01,   1.3828e+00,  -7.2477e-01]])

In [76]:
arr = np.zeros((4, 4))
arr_3d = arr[:, np.newaxis, :]
arr_3d.shape

(4, 1, 4)

In [82]:
arr_1d = np.random.normal(size=3)
arr_1d
arr_1d[:, np.newaxis]

array([-2.3702, -1.8608, -0.8608])

array([[-2.3702],
       [-1.8608],
       [-0.8608]])

In [83]:
arr_1d[np.newaxis, :]

array([[-2.3702, -1.8608, -0.8608]])

In [86]:
arr = randn(3, 4, 5)
arr
depth_means = arr.mean(2)
depth_means
demeaned = arr - depth_means[:, :, np.newaxis]
demeaned
demeaned.mean(2)

array([[[-1.5637, -0.3675,  1.0459,  1.22  , -0.2477],
        [-0.4162, -0.1167, -1.8448,  2.0687, -0.777 ],
        [ 1.4402, -0.1106,  1.2274,  1.9208,  0.7464],
        [ 2.2247, -0.6794,  0.7274, -0.8687, -1.2139]],

       [[-0.4706, -0.9192, -0.8388,  0.4352, -0.5578],
        [-0.5675, -0.3726, -0.9266,  1.7551,  1.2098],
        [ 1.27  , -0.9744, -0.6347, -0.3957, -0.2894],
        [-0.7343, -0.7285,  0.8388,  0.2669,  0.7212]],

       [[ 0.911 , -1.0209, -1.4134,  1.2966,  0.2523],
        [ 1.1275, -0.5684,  0.3094, -0.5774, -1.1686],
        [-0.825 , -2.6444, -0.153 , -0.7519, -0.1326],
        [ 1.4573,  0.6095, -0.4938,  1.24  , -0.1357]]])

array([[ 0.0174, -0.2172,  1.0448,  0.038 ],
       [-0.4703,  0.2197, -0.2048,  0.0728],
       [ 0.0051, -0.1755, -0.9014,  0.5355]])

array([[[ -1.5811e+00,  -3.8488e-01,   1.0285e+00,   1.2026e+00,
          -2.6509e-01],
        [ -1.9903e-01,   1.0046e-01,  -1.6276e+00,   2.2859e+00,
          -5.5976e-01],
        [  3.9532e-01,  -1.1554e+00,   1.8254e-01,   8.7594e-01,
          -2.9841e-01],
        [  2.1867e+00,  -7.1741e-01,   6.8936e-01,  -9.0674e-01,
          -1.2519e+00]],

       [[ -3.6119e-04,  -4.4897e-01,  -3.6856e-01,   9.0543e-01,
          -8.7535e-02],
        [ -7.8711e-01,  -5.9229e-01,  -1.1462e+00,   1.5355e+00,
           9.9016e-01],
        [  1.4749e+00,  -7.6954e-01,  -4.2987e-01,  -1.9086e-01,
          -8.4596e-02],
        [ -8.0711e-01,  -8.0132e-01,   7.6596e-01,   1.9408e-01,
           6.4838e-01]],

       [[  9.0587e-01,  -1.0260e+00,  -1.4185e+00,   1.2915e+00,
           2.4717e-01],
        [  1.3030e+00,  -3.9286e-01,   4.8487e-01,  -4.0188e-01,
          -9.9313e-01],
        [  7.6369e-02,  -1.7430e+00,   7.4840e-01,   1.4947e-01,
           7.6878e-01],
        [  9.2184

array([[  0.0000e+00,   6.6613e-17,   0.0000e+00,   8.8818e-17],
       [ -2.2204e-17,   4.4409e-17,  -1.1102e-17,   2.2204e-17],
       [ -5.5511e-17,   0.0000e+00,  -2.2204e-17,   0.0000e+00]])

In [87]:
def demean_axis(arr, axis=0):
    mean = arr.mean(axis)
    
    indexer = [slice(None)] * arr.ndim
    indexer[axis] = np.newaxis
    return arr - means[indexer]

## Setting array values by broadcasting

In [90]:
arr = np.zeros((4, 3))
arr
arr[:] = 5
arr

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

array([[ 5.,  5.,  5.],
       [ 5.,  5.,  5.],
       [ 5.,  5.,  5.],
       [ 5.,  5.,  5.]])

In [93]:
col = np.array([1.28, -0.42, 0.44, 1.6])
arr[:] = col[:, np.newaxis]
arr

array([[ 1.28,  1.28,  1.28],
       [-0.42, -0.42, -0.42],
       [ 0.44,  0.44,  0.44],
       [ 1.6 ,  1.6 ,  1.6 ]])

In [96]:
arr[:2]

array([[-1.37 , -1.37 , -1.37 ],
       [ 0.509,  0.509,  0.509]])

In [97]:
arr[:2] = [[-1.37], [0.509]]
arr

array([[-1.37 , -1.37 , -1.37 ],
       [ 0.509,  0.509,  0.509],
       [ 0.44 ,  0.44 ,  0.44 ],
       [ 1.6  ,  1.6  ,  1.6  ]])

## Concatenating along an axis

In [98]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [99]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [101]:
s1 = Series([0, 1], index=['a', 'b'])
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = Series([5, 6], index=['f', 'g'])

s1
s2
s3

a    0
b    1
dtype: int64

c    2
d    3
e    4
dtype: int64

f    5
g    6
dtype: int64

In [102]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [103]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [105]:
s4 = pd.concat([s1 * 5, s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [106]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [107]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,5


In [108]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


In [110]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [111]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [112]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [113]:
df1 = DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                columns=['one', 'two'])
df2 = DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                columns=['three', 'four'])
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [114]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [115]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [116]:
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

df1
df2

,a,b,c,d
0,1.430042,-0.846852,0.603282,1.263572
1,-0.255491,-0.445688,0.468367,-0.961604
2,-1.824505,0.625428,1.022872,1.107425


,b,d,a
0,0.090937,-0.350109,0.217957
1,-0.894813,-1.741494,-1.052256


In [117]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.430042,-0.846852,0.603282,1.263572
1,-0.255491,-0.445688,0.468367,-0.961604
2,-1.824505,0.625428,1.022872,1.107425
3,0.217957,0.090937,NaN,-0.350109
4,-1.052256,-0.894813,NaN,-1.741494


## Combining data with overlap

In [118]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b = Series(np.arange(len(a), dtype=np.float64),
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan

a
b

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [119]:
np.where(pd.isnull(a), b, a)

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

In [120]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [122]:
df1 = DataFrame({'a': [1., np.nan, 5., np.nan],
                 'b': [np.nan, 2., np.nan, 6.],
                 'c': range(2, 18, 4)})
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.],
                 'b': [np.nan, 3., 4., 6., 8.]})

df1
df2
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## Reshaping and pivoting

### Reshaping with hierarchical indexing

In [123]:
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [124]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [125]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [126]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [127]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [130]:
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])
s1
s2
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2
data2.unstack()

a    0
b    1
c    2
d    3
dtype: int64

c    4
d    5
e    6
dtype: int64

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [129]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [131]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [132]:
df = DataFrame({'left': result, 'right': result + 5},
               columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [133]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [134]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

## Pivoting "long" to "wide" format

In [139]:
data = pd.read_csv('ch07/macrodata.csv')
data

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
5,1960.0,2.0,2834.390,1792.9,298.152,460.400,1966.1,29.550,140.2,2.68,5.2,180.671,0.14,2.55
6,1960.0,3.0,2839.022,1785.8,296.375,474.676,1967.8,29.750,140.9,2.36,5.6,181.528,2.70,-0.34
7,1960.0,4.0,2802.616,1788.2,259.764,476.434,1966.6,29.840,141.1,2.29,6.3,182.287,1.21,1.08
8,1961.0,1.0,2819.264,1787.7,266.405,475.854,1984.5,29.810,142.1,2.37,6.8,182.992,-0.40,2.77
9,1961.0,2.0,2872.005,1814.3,286.246,480.328,2014.4,29.920,142.9,2.29,7.0,183.691,1.47,0.81


In [140]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [141]:
data = DataFrame(data.to_records(),
                 columns=pd.Index(['realgdp', 'infl', 'unemp'], name='item'),
                 index=periods.to_timestamp('D', 'end'))
data

item,realgdp,infl,unemp
date,,,
1959-03-31,2710.349,0.00,5.8
1959-06-30,2778.801,2.34,5.1
1959-09-30,2775.488,2.74,5.3
1959-12-31,2785.204,0.27,5.6
1960-03-31,2847.699,2.31,5.2
1960-06-30,2834.390,0.14,5.2
1960-09-30,2839.022,2.70,5.6
1960-12-31,2802.616,1.21,6.3
1961-03-31,2819.264,-0.40,6.8


In [144]:
data.stack()

date        item   
1959-03-31  realgdp     2710.349
            infl           0.000
            unemp          5.800
1959-06-30  realgdp     2778.801
            infl           2.340
            unemp          5.100
1959-09-30  realgdp     2775.488
            infl           2.740
            unemp          5.300
1959-12-31  realgdp     2785.204
            infl           0.270
            unemp          5.600
1960-03-31  realgdp     2847.699
            infl           2.310
            unemp          5.200
1960-06-30  realgdp     2834.390
            infl           0.140
            unemp          5.200
1960-09-30  realgdp     2839.022
            infl           2.700
            unemp          5.600
1960-12-31  realgdp     2802.616
            infl           1.210
            unemp          6.300
1961-03-31  realgdp     2819.264
            infl          -0.400
            unemp          6.800
1961-06-30  realgdp     2872.005
            infl           1.470
            unemp      

In [146]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata[:10]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


In [143]:
wdata = ldata.pivot('date', 'item', 'value')
wdata

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2
1960-06-30,0.14,2834.390,5.2
1960-09-30,2.70,2839.022,5.6
1960-12-31,1.21,2802.616,6.3
1961-03-31,-0.40,2819.264,6.8


In [148]:
ldata.set_index(['date', 'item'])

value
date       item              
1959-03-31 realgdp   2710.349
           infl         0.000
           unemp        5.800
1959-06-30 realgdp   2778.801
           infl         2.340
           unemp        5.100
1959-09-30 realgdp   2775.488
           infl         2.740
           unemp        5.300
1959-12-31 realgdp   2785.204
           infl         0.270
           unemp        5.600
1960-03-31 realgdp   2847.699
           infl         2.310
           unemp        5.200
1960-06-30 realgdp   2834.390
           infl         0.140
           unemp        5.200
1960-09-30 realgdp   2839.022
           infl         2.700
           unemp        5.600
1960-12-31 realgdp   2802.616
           infl         1.210
           unemp        6.300
1961-03-31 realgdp   2819.264
           infl        -0.400
           unemp        6.800
1961-06-30 realgdp   2872.005
           infl         1.470
           unemp        7.000
1961-09-30 realgdp   2918.419
           infl         0.800
           unemp        6.800
1961-12-31 realgdp   2977.830
           infl         0.800
           unemp        6.200
1962-03-31 realgdp   3031.241
           infl         2.260
           unemp        5.600
1962-06-30 realgdp   3064.709
           infl         0.130
           unemp        5.500
1962-09-30 realgdp   3093.047
           infl         2.110
           unemp        5.600
1962-12-31 realgdp   3100.563
           infl         0.790
           unemp        5.500
1963-03-31 realgdp   3141.087
           infl         0.530
...                       ...
2005-09-30 infl         9.140
           unemp        5.000
2005-12-31 realgdp  12748.699
           infl         0.400
           unemp        4.900
2006-03-31 realgdp  12915.938
           infl         2.600
           unemp        4.700
2006-06-30 realgdp  12962.462
           infl         3.970
           unemp        4.700
2006-09-30 realgdp  12965.916
           infl        -1.580
           unemp        4.700
2006-12-31 realgdp  13060.679
           infl         3.300
           unemp        4.400
2007-03-31 realgdp  13099.901
           infl         4.580
           unemp        4.500
2007-06-30 realgdp  13203.977
           infl         2.750
           unemp        4.500
2007-09-30 realgdp  13321.109
           infl         3.450
           unemp        4.700
2007-12-31 realgdp  13391.249
           infl         6.380
           unemp        4.800
2008-03-31 realgdp  13366.865
           infl         2.820
           unemp        4.900
2008-06-30 realgdp  13415.266
           infl         8.530
           unemp        5.400
2008-09-30 realgdp  13324.600
           infl        -3.160
           unemp        6.000
2008-12-31 realgdp  13141.920
           infl        -8.790
           unemp        6.900
2009-03-31 realgdp  12925.410
           infl         0.940
           unemp        8.100
2009-06-30 realgdp  12901.504
           infl         3.370
           unemp        9.200
2009-09-30 realgdp  12990.341
           infl         3.560
           unemp        9.600

[609 rows x 1 columns]

In [149]:
ldata[:10]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


In [150]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


In [151]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:5]

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,1.436603
1,1959-03-31,infl,0.000,-0.576207
2,1959-03-31,unemp,5.800,-2.420294
3,1959-06-30,realgdp,2778.801,-1.062330
4,1959-06-30,infl,2.340,0.237372


In [152]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.576207  1.436603 -2.420294
1959-06-30  2.34  2778.801   5.1  0.237372 -1.062330  0.000957
1959-09-30  2.74  2775.488   5.3 -1.367524  0.065253 -0.030280
1959-12-31  0.27  2785.204   5.6 -0.642437  0.940489  1.040179
1960-03-31  2.31  2847.699   5.2  0.429214 -1.082922 -0.236224

In [153]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


In [154]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.576207  1.436603 -2.420294
1959-06-30  2.34  2778.801   5.1  0.237372 -1.062330  0.000957
1959-09-30  2.74  2775.488   5.3 -1.367524  0.065253 -0.030280
1959-12-31  0.27  2785.204   5.6 -0.642437  0.940489  1.040179
1960-03-31  2.31  2847.699   5.2  0.429214 -1.082922 -0.236224
1960-06-30  0.14  2834.390   5.2 -0.331661  0.641818  1.394072
1960-09-30  2.70  2839.022   5.6 -0.192466 -1.076742 -0.871188

## Data transformation

### Removing duplicates

In [155]:
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [157]:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [158]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [159]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3


In [162]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
6,two,4,6


## Transforming data using a function or mapping

In [163]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
                           'corned beef', 'Bacon', 'pastrami', 'honey ham',
                           'nova lox'],
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [165]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}
meat_to_animal

{'bacon': 'pig',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon',
 'pastrami': 'cow',
 'pulled pork': 'pig'}

In [166]:
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [170]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [171]:
%timeit data['food'].map(lambda x: meat_to_animal[x.lower()])

10000 loops, best of 3: 116 µs per loop


## Replacing values

In [172]:
data = Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [173]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [174]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [175]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [176]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

## Renaming axis indexes

In [178]:
data = DataFrame(np.arange(12).reshape((3, 4)),
                index=['Ohio', 'Colorado', 'New York'],
                columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [179]:
data.index.map(str.upper)

Index(['OHIO', 'COLORADO', 'NEW YORK'], dtype='object')

In [180]:
data.index = data.index.map(str.upper)
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [181]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [182]:
data.rename(index={'OHIO' : 'INDIANA'},
           columns={'three': 'peekaboo'})
#원본데이터는 변경되지 않음

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [183]:
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [184]:
_ = data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


## Discretization and binning

In [188]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [189]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [190]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [191]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [192]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [193]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [MiddleAged < Senior < YoungAdult < Youth]

In [194]:
pd.value_counts(pd.cut(ages, bins, labels=group_names))

Youth         5
YoungAdult    3
MiddleAged    3
Senior        1
dtype: int64

In [197]:
data = np.random.rand(20)
data
pd.cut(data, 4, precision=2)

array([ 0.1853,  0.2201,  0.2955,  0.7333,  0.4926,  0.966 ,  0.651 ,
        0.2008,  0.279 ,  0.3593,  0.0592,  0.2681,  0.3987,  0.6239,
        0.623 ,  0.9879,  0.8179,  0.3249,  0.0743,  0.7706])

[(0.058, 0.29], (0.058, 0.29], (0.29, 0.52], (0.52, 0.76], (0.29, 0.52], ..., (0.76, 0.99], (0.76, 0.99], (0.29, 0.52], (0.058, 0.29], (0.76, 0.99]]
Length: 20
Categories (4, interval[float64]): [(0.058, 0.29] < (0.29, 0.52] < (0.52, 0.76] < (0.76, 0.99]]

In [198]:
catal = pd.cut(data, 5)
data
catal

array([ 0.1853,  0.2201,  0.2955,  0.7333,  0.4926,  0.966 ,  0.651 ,
        0.2008,  0.279 ,  0.3593,  0.0592,  0.2681,  0.3987,  0.6239,
        0.623 ,  0.9879,  0.8179,  0.3249,  0.0743,  0.7706])

[(0.0582, 0.245], (0.0582, 0.245], (0.245, 0.431], (0.616, 0.802], (0.431, 0.616], ..., (0.802, 0.988], (0.802, 0.988], (0.245, 0.431], (0.0582, 0.245], (0.616, 0.802]]
Length: 20
Categories (5, interval[float64]): [(0.0582, 0.245] < (0.245, 0.431] < (0.431, 0.616] < (0.616, 0.802] < (0.802, 0.988]]

In [199]:
pd.value_counts(catal)

(0.245, 0.431]     6
(0.616, 0.802]     5
(0.0582, 0.245]    5
(0.802, 0.988]     3
(0.431, 0.616]     1
dtype: int64

In [200]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [201]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(0.174, 0.379], (0.174, 0.379], (0.174, 0.379], (0.379, 0.833], (0.379, 0.833], ..., (0.833, 0.988], (0.379, 0.833], (0.174, 0.379], (0.0582, 0.174], (0.379, 0.833]]
Length: 20
Categories (4, interval[float64]): [(0.0582, 0.174] < (0.174, 0.379] < (0.379, 0.833] < (0.833, 0.988]]

## Detecting and filtering outliers

In [202]:
np.random.seed(12345)
data = DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [203]:
col = data[3]
col[np.abs(col) > 3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [204]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [205]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


## Permutation and random sampling

In [206]:
df = DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([1, 0, 2, 3, 4])

In [207]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [208]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [209]:
df.take(np.random.permutation(len(df))[:3])

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
4,16,17,18,19


In [211]:
bag = np.array([5, 7, -1, 6, 4])
sampler = np.random.randint(0, len(bag), size=10)
sampler

array([1, 2, 3, 0, 1, 2, 2, 3, 2, 1])

In [212]:
draws = bag.take(sampler)
draws

array([ 7, -1,  6,  5,  7, -1, -1,  6, -1,  7])

In [213]:
bag[sampler]

array([ 7, -1,  6,  5,  7, -1, -1,  6, -1,  7])

## Lotto Simulation

In [259]:
lotto = np.random.permutation(45)[:6] + 1
lotto # 1게임 진행
lottos = Series(np.arange(5))\
               .map(lambda x: np.sort(np.random.permutation(45)[:6] + 1))
# 5게임 진행 code
lottos

array([38, 21, 33, 18,  8, 19])

0     [3, 16, 17, 21, 35, 41]
1    [17, 18, 22, 26, 28, 41]
2      [3, 5, 10, 17, 18, 26]
3       [2, 3, 5, 28, 29, 45]
4    [15, 24, 25, 29, 43, 44]
dtype: object

In [253]:
for i in range(5):
    lotto = np.random.permutation(45)[:6]+1
    print (lotto)

[ 1 37 33  3 15 36]
[32 42 26  7 39 33]
[35 17 37 32 13 36]
[28  8 38 24 32 43]
[ 2 10 15 43  3  9]


## Computing indicator / dummy variables

In [260]:
df= DataFrame({'key' : ['b', 'b', 'a', 'c', 'a', 'b'],
              'data1' : range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [262]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [263]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ch02/movielens/movies.dat', sep='::', header=None,
                        names=mnames)
movies[:10]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [269]:
genre_iter = (set(x.split('|')) for x in movies.genres)
genres = sorted(set.union(*genre_iter))
genres

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [272]:
dummies = DataFrame(np.zeros((len(movies), len(genres))), columns=genres)
dummies[:10]

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [275]:
for i, gen in enumerate(movies.genres):
    dummies.loc[i, gen.split('|')] = 1
dummies

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [278]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.loc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Action                                   0
Genre_Adventure                                0
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Crime                                    0
Genre_Documentary                              0
Genre_Drama                                    0
Genre_Fantasy                                  0
Genre_Film-Noir                                0
Genre_Horror                                   0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Romance                                  0
Genre_Sci-Fi                                   0
Genre_Thriller                                 0
Genre_War                                      0
Genre_Western       

In [282]:
np.random.seed(12345)

In [283]:
values = np.random.rand(10)
values

array([ 0.9296,  0.3164,  0.1839,  0.2046,  0.5677,  0.5955,  0.9645,
        0.6532,  0.7489,  0.6536])

In [284]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins)).sum()

(0.0, 0.2]    1
(0.2, 0.4]    2
(0.4, 0.6]    2
(0.6, 0.8]    3
(0.8, 1.0]    2
dtype: int64

# String manipulation

## string object methods

In [285]:
val = 'a,b, guido'
val.split(',')

['a', 'b', ' guido']

In [287]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [289]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [290]:
'::'.join(pieces)

'a::b::guido'

In [291]:
'guido' in val

True

In [292]:
val.index(',')

1

In [293]:
val.find(':')

-1

In [294]:
val.index(':')

ValueError: substring not found

In [295]:
val.replace(',', '::')

'a::b:: guido'

In [296]:
val.replace(',', '')

'ab guido'

## Recular expression

In [297]:
import re
text = "foo    bar\t baz  \tqux"
[x.split]
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [298]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [299]:
regex.findall(text)

['    ', '\t ', '  \t']

In [302]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'


regex = re.compile(pattern, flags=re.IGNORECASE)
text
pattern
regex

'Dave dave@google.com\nSteve steve@gmail.com\nRob rob@gmail.com\nRyan ryan@yahoo.com\n'

'[A-Z0-9._%+-]+@[A-Z0-9.-]+\\.[A-Z]{2,4}'

re.compile(r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}', re.IGNORECASE|re.UNICODE)

In [304]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [308]:
m = regex.search(text)
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [309]:
text[m.start():m.end()]

'dave@google.com'